# Parsing PDF's with Regular Expression

This project goes over parsing Pdf's with regular expressions, also know as regex. Data stored in pdfs can be difficult to import into a database, but since pdfs are actually text, regex patterns can be used to collect data. This project will go over using regex on a pdf to turn the pdf into a dataframe, and then insert it into a SQL server.

## Summary of the Dataset

The pdf I will be parsing is from the Missouri Department of Health & Senior Services website: https://health.mo.gov/safety/healthservregs/pdf/MOhospByName.pdf. The pdf does not have any available .txt or .csv file for download. Therefore, I will use regex to collect the data, because there is a pattern.

### Import pdf to Python

The first step is to use the requests library in Python to grab the url and write it as a .pdf file for record keeping.

In [15]:
import requests


url_data ='https://health.mo.gov/safety/healthservregs/pdf/MOhospByName.pdf'

MO_data = requests.get(url_data, allow_redirects=True , verify=True)


open(r"C:/Users/blong/Desktop/Python/MO-hosp-data.pdf", "wb").write(MO_data.content)

178876

### Convert pdf to text

The next step includes loading the other libraries needed: re for regex, pandas, and pdfplumber. Pdfplummer was the library I used to convert the pdf into text. There are other libraries similar to pdfplummer, but I personally found that this one was the most effective.

In [2]:
import pdfplumber
import re
import pandas as pd

pdf = pdfplumber.open(r"C:/Users/blong/Desktop/Python/MO-hosp-data.pdf")

### Explore first page of the pdf

With pdfplumber, the text can be extracted from each page. This is the first page and all the information it contains.

In [3]:
text = pdf.pages[0].extract_text()
text

'MO Hospital Profiles By Name Total Fac Count Last updated\n162 Friday, January 8, 2021\nFacility ID: H006 Barnes-Jewish Hospital Licensed Beds by Classification\nSwingbeds? One Barnes Jewish Hospital Plaza Med/Surg Beds: 677 LTC beds: 0\nCritical Access? St. Louis, MO 63110 Pediatric Beds: 0 ICU Beds: 173\nPPS rehab unit? Phone: (314) 747-3000 County ST LOUIS CIT Alcohol/Drug Beds 0 OB Beds: 0\nPPS psych unit?\nAccredited? Fax: (314) 362-0468 Rehab Beds: 0 Psych Beds: 46\nby JointCom Administrator: Dr. John Lynch, President NICU Beds: 0 Total Beds Lic: 896\nCMS Provider#: 260032 Hospital  Type: General Acute Care Hospital\nLicense #:421 License Exp 01/31/2021\nFacility ID: H005 Barnes-Jewish Hospital - North Licensed Beds by Classification\nSwingbeds? One Parkview Place Med/Surg Beds: 380 LTC beds: 0\nCritical Access? St. Louis, MO 63110 Pediatric Beds: 0 ICU Beds: 32\nPPS rehab unit? Phone: (314) 747-3000 County ST LOUIS CIT Alcohol/Drug Beds 0 OB Beds: 70\nPPS psych unit?\nAccredite

### Resources for Regex
I am using the re library to primarily the find all functions and will demonstrate examples of regex. Since Regular Expressions can be tricky, I linked a couple resourceful websites that will be helpful for the project.

https://www.w3schools.com/python/python_regex.asp
https://www.debuggex.com/cheatsheet/regex/python

### Examples of Regex

This might seem like a wall of text, but in between all the text there are patterns that can be extracted. I will go over some of the regex I used in this project.

In [4]:
re.findall('Facility ID: ([A-Z][0-9]{3})', text)

['H006', 'H005', 'H177', 'H007', 'H008', 'H010']

First, when looking for a facility ID it is always preceded by "Facility ID: ". Thus using () to group all the text afterwards. Since all the IDs are one capital letter followed by the numbers, regex will be great for this. [A-Z] defines as finding capital letters, [0-9] defines as finding any numbers, and {3} defines as finding only 3 numbers.

In [5]:
re.findall('Swingbeds\?\ (.*) Med/Surg', text)

['One Barnes Jewish Hospital Plaza',
 'One Parkview Place',
 '5355 Delmar Blvd',
 '#10 Hospital Drive',
 '12634 Olive Boulevard',
 '615 West Nursery Street, P.O. Box 370']

The addresses is always located between "Swingbeds? " and "Med/Surg". Use the () to group the text in between those two items. Since address can vary a lot in size, numbers and letters. (.*) is used to find any character. There is a slash in between ? (\?\), because ? is a regex character, and without the backslashes will add a confusion to regex.

In [6]:
re.findall('Fax:(^ $|.*) Rehab', text)

[' (314) 362-0468',
 ' (314) 362-0468',
 '',
 ' (636) 916-9414',
 ' (314) 996-8619',
 ' (660) 679-4381']

Another example is when there are blank values as in case with fax. Here the ^ and $, which means starts with and ends with, and in this case a blank space " ". Now since there are fax numbers the | is used as or. That way spaces or numbers will be collected.

In [7]:
re.findall('[^N]ICU Beds:(^$|.*)', text)

[' 173', ' 32', ' 0', ' 12', ' 4', ' 6']

Here is another good example because it contains NICU and ICU. If just ICU is put as the text qualifier before the group, it will pick up NICU as well since it includes that so the [^N] is excluding the N. Also ^$ this time has no space since there is actually no space in the text to pick up empty values. Thus it is always important to double check how the regex is written.

### Putting everything together

Now I'll be combining all the different regex to collect all the pieces of wanted data from the pdf. First an empty list will have to be used since each page needs to be appended. Then a for loop will run through each page of the pdf which is a total of 24 pages.

In [8]:
text_all , facid_all, cms_all, facname_all, address_all, town_all, state_all, zip_all, phone_all, county_all, fax_all, admin_all, type_all, medsurg_all, ped_all, drug_all, rehab_all, NICU_all, LTC_all, ICU_all, OB_all, pysch_all, totalbeds_all,  license_all, licensedate_all  = ([] for i in range(25))

setnum = [i for i in range(24)]

for i in setnum :
    text = pdf.pages[i].extract_text()
    text_all.append(text)
    text_facid = re.findall('Facility ID: ([A-Z][0-9]{3})', text_all[i])
    facid_all.append(text_facid)
    text_cms = re.findall('CMS Provider#: (.*) Hospital', text_all[i])
    cms_all.append(text_cms)    
    text_facname = re.findall('[A-Z][0-9]{3} (.*) Licensed', text_all[i])
    facname_all.append(text_facname)  
    text_address = re.findall('Swingbeds\?\ (.*) Med/Surg', text_all[i])
    address_all.append(text_address) 
    text_town = re.findall('Critical Access\?\ (.*),', text_all[i])
    town_all.append(text_town)
    text_state = re.findall('[A-z]+, ([A-Z]{2}) [0-9]+', text_all[i])
    state_all.append(text_state)
    text_zip = re.findall('[A-z]{2} ([0-9]+) Pediatric', text_all[i])
    zip_all.append(text_zip)
    text_phone = re.findall('Phone: (.*) County', text_all[i])
    phone_all.append(text_phone)
    text_fax = re.findall('Fax:(^ $|.*) Rehab', text_all[i]) 
    fax_all.append(text_fax)
    text_county = re.findall('County (.*) Alcohol/Drug', text_all[i])
    county_all.append(text_county)
    text_admin = re.findall('Administrator: (.*) NICU', text_all[i])
    admin_all.append(text_admin)
    text_type = re.findall('Hospital  Type:(^$|.*)', text_all[i]) 
    type_all.append(text_type)
    text_medsurg =  re.findall('Med/Surg Beds:(^$|.*) LTC', text_all[i])
    medsurg_all.append(text_medsurg)
    text_ped =  re.findall('Pediatric Beds: ([0-9]+)', text_all[i]) 
    ped_all.append(text_ped)
    text_drug =  re.findall('Alcohol/Drug Beds ([0-9]+)', text_all[i]) 
    drug_all.append(text_drug)
    text_rehab =  re.findall('Rehab Beds: ([0-9]+)', text_all[i]) 
    rehab_all.append(text_rehab)
    text_NICU =  re.findall('NICU Beds: ([0-9]+)', text_all[i]) 
    NICU_all.append(text_NICU)
    text_LTC =  re.findall('LTC beds:(^$|.*)', text_all[i]) 
    LTC_all.append(text_LTC)
    text_ICU =  re.findall('[^N]ICU Beds:(^$|.*)', text_all[i])
    ICU_all.append(text_ICU)
    text_OB =  re.findall('OB Beds: ([0-9]+)', text_all[i]) 
    OB_all.append(text_OB)
    text_pysch =  re.findall('Psych Beds: ([0-9]+)', text_all[i]) 
    pysch_all.append(text_pysch)
    text_totalbeds =  re.findall('Total Beds Lic: ([0-9]+)', text_all[i]) 
    totalbeds_all.append(text_totalbeds)
    text_license =  re.findall('License #:(^$|.*) License', text_all[i])
    license_all.append(text_license)
    text_licensedate =  re.findall('License Exp(^$|.*)', text_all[i])
    licensedate_all.append(text_licensedate)    
    

### Convert from list to series

Next, each list of data needs to be converted to a series before it can be put into a dataframe.

In [9]:
facid_all = pd.concat([ pd.Series(x) for x in facid_all ] , ignore_index=True)
cms_all = pd.concat([ pd.Series(x) for x in cms_all ] , ignore_index=True)
facname_all = pd.concat([ pd.Series(x) for x in facname_all ] , ignore_index=True)
address_all = pd.concat([ pd.Series(x) for x in address_all ] , ignore_index=True)
town_all = pd.concat([ pd.Series(x) for x in town_all ] , ignore_index=True)
state_all = pd.concat([ pd.Series(x) for x in state_all ] , ignore_index=True)
zip_all = pd.concat([ pd.Series(x) for x in zip_all ] , ignore_index=True)
phone_all = pd.concat([ pd.Series(x) for x in phone_all ] , ignore_index=True)
fax_all = pd.concat([ pd.Series(x) for x in fax_all ] , ignore_index=True)
county_all = pd.concat([ pd.Series(x) for x in county_all ] , ignore_index=True)
admin_all = pd.concat([ pd.Series(x) for x in admin_all ] , ignore_index=True)
type_all = pd.concat([ pd.Series(x) for x in type_all ] , ignore_index=True)
medsurg_all = pd.concat([ pd.Series(x) for x in medsurg_all ] , ignore_index=True)  
ped_all = pd.concat([ pd.Series(x) for x in ped_all ] , ignore_index=True)
drug_all = pd.concat([ pd.Series(x) for x in drug_all ] , ignore_index=True)
rehab_all = pd.concat([ pd.Series(x) for x in rehab_all ] , ignore_index=True)
NICU_all = pd.concat([ pd.Series(x) for x in NICU_all ] , ignore_index=True)
LTC_all = pd.concat([ pd.Series(x) for x in LTC_all ] , ignore_index=True)
ICU_all = pd.concat([ pd.Series(x) for x in ICU_all ] , ignore_index=True)
OB_all = pd.concat([ pd.Series(x) for x in OB_all ] , ignore_index=True)
pysch_all = pd.concat([ pd.Series(x) for x in pysch_all ] , ignore_index=True)
totalbeds_all = pd.concat([ pd.Series(x) for x in totalbeds_all ] , ignore_index=True)
license_all = pd.concat([ pd.Series(x) for x in license_all ] , ignore_index=True)
licensedate_all = pd.concat([ pd.Series(x) for x in licensedate_all ] , ignore_index=True)


### Create data frame

Now, after all the fields that have been created can be put into a pandas data frame, it also needs to be transposed. After that, name the columns with their appropriate title.

In [10]:
data_frame = pd.DataFrame(( facid_all , cms_all , facname_all , address_all, 
                    town_all, state_all, zip_all, phone_all,
                   fax_all, county_all, admin_all, type_all, 
                   medsurg_all, ped_all, drug_all, rehab_all, 
                   NICU_all, LTC_all , ICU_all, OB_all, 
                   pysch_all, totalbeds_all, license_all, 
                   licensedate_all ) ).T


data_frame.columns = ['FacID','CMS','Facility','Address','Town' ,'State','Zip','Phone','Fax','County'
              ,'Admin','Type','Med_Surg','Peds','Drug','Rehab','NICU','LTC','ICU','OB'
              ,'Pysch','Total_Beds','License_Number','License_exp_date']



### Clean up data frame

Just remove the first spaces that were left over and it's all done! From pdf to text to dataframe!

In [11]:
data_frame['Fax'] = data_frame['Fax'].str[1:]
data_frame['Type'] = data_frame['Type'].str[1:]
data_frame['Med_Surg'] = data_frame['Med_Surg'].str[1:]
data_frame['LTC'] = data_frame['LTC'].str[1:]
data_frame['ICU'] = data_frame['ICU'].str[1:]
data_frame['License_Number'] = data_frame['License_Number'].str[1:]
data_frame['License_exp_date'] = data_frame['License_exp_date'].str[1:]

data_frame

,FacID,CMS,Facility,Address,Town,State,Zip,Phone,Fax,County,...,Drug,Rehab,NICU,LTC,ICU,OB,Pysch,Total_Beds,License_Number,License_exp_date
0,H006,260032 Hospital Type: General Acute Care,Barnes-Jewish Hospital,One Barnes Jewish Hospital Plaza,St. Louis,MO,63110,(314) 747-3000,(314) 362-0468,ST LOUIS CIT,...,0,0,0,0,173,0,46,896,21,01/31/2021
1,H005,260032 Hospital Type: General Acute Care,Barnes-Jewish Hospital - North,One Parkview Place,St. Louis,MO,63110,(314) 747-3000,(314) 362-0468,ST LOUIS CIT,...,0,0,0,0,32,70,0,482,21,01/31/2021
2,H177,26S032 Hospital Type: Psychiatric,Barnes-Jewish Hospital - Psychiatric Su,5355 Delmar Blvd,St. Louis,MO,63112,(314) 514-3535,,ST LOUIS CIT,...,0,0,0,0,0,0,50,50,21,01/31/2021
3,H007,260191 Hospital Type: General Acute Care,Barnes-Jewish St. Peters Hospital,#10 Hospital Drive,St. Peters,MO,63376,(636) 916-9000,(636) 916-9414,ST CHARLES,...,0,0,0,0,12,0,0,102,57,08/31/2021
4,H008,260162 Hospital Type: General Acute Care,Barnes-Jewish West County Hospital,12634 Olive Boulevard,Creve Coeur,MO,63141,(314) 996-8000,(314) 996-8619,ST LOUIS,...,0,0,0,0,4,0,0,68,68,10/31/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,H151,260141 Hospital Type: General Acute Care,University of Missouri Health Care,"One Hospital Drive, Room CE121, DC031.00",Columbia,MO,65212,(573) 882-4141,(573) 884-4174,BOONE,...,0,0,0,0,0,0,0,0,,
158,H142,261308 Hospital Type: General Acute Care,Washington County Memorial Hospital,300 Health Way,Potosi,MO,63664,(573) 438-5451,(573) 438-2399,WASHINGTON,...,0,0,0,0,0,0,0,25,24,11/30/2021
159,H143,260097 Hospital Type: General Acute Care,Western Missouri Medical Center,403 Burkarth Road,Warrensburg,MO,64093,(660) 747-2500,(660) 747-8455,JOHNSON,...,0,0,0,0,8,18,12,62,21,07/31/2021
160,H146,260141 Hospital Type: General Acute Care,Women's and Children's Hospital,404 Keene Street,Columbia,MO,65201,(573) 875-9200,(573) 875-9869,BOONE,...,0,0,0,0,0,0,0,0,,


## Inserting into SQL Server

Here I will give the steps to inserting the data frame into a MS SQL Server since I don't want to actually run it. I will commit the code out, but it will work.

### Connect to Sever

In [12]:
#import pyodbc
 
#conn = pyodbc.connect('Driver={SQL Server};' 
#    'Server=Aarons_awesome_server ;'
#    'Database=facility_info;'
#    'Trusted_Connection=no;'
#    'UID=Buy_ETH;'
#    'PWD=password123;')

#cursor = conn.cursor()

### Drop table and create table

Drop the table in case data is already in it. Now create the table with the various names and data types.

In [13]:
# cursor.execute('''DROP TABLE MO_DHSS_Fac_List''')
# conn.commit()


# cursor.execute('''
#                 CREATE TABLE MO_DHSS_Fac_List
#                     (
#                     FacID nchar(10),
#                     CMS nchar(100),
#                     Facility nchar(100),
#                     Address nchar(100),
#                     Town nchar(100),
#                     State nchar(100),
#                     Zip nchar(20),
#                     Phone nchar(100),
#                     Fax nchar(100),
#                     Admin nchar(100),
#                     Type nchar(100),
#                     Med_Surg int,
#                     Peds int,
#                     Drug int,
#                     Rehab int,
#                     NICU int,
#                     LTC int,
#                     ICU int,
#                     OB int,
#                     Pysch int,
#                     Total_Beds int,
#                     License_Number int,
#                     License_exp_date nchar(100)
#                     )
#                 ''')
# conn.commit()

### Load in the data

In [14]:
# count = 0
# for index,row in data_frame.iterrows():
#     cursor.execute('''INSERT INTO [facility_info].[dbo].[MO_DHSS_Hosp_List]([FacID],
#                    [CMS],[Facility],[Address],[Town],
#                    [State],[Zip],[Phone],[Fax],[Admin],[Type],
#                    [Med_Surg],[Peds],[Drug],[Rehab],[NICU],[LTC],[ICU],[OB],[Pysch],
#                    [Total_Beds],[License_Number],[License_exp_date])
#                     values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', 
#                     row['FacID'],row['CMS'],row['Facility'],row['Address'],row['Town'],
#                     row['State'],row['Zip'],row['Phone'],
#                     row['Fax'],row['Admin'],row['Type'],
#                     row['Med_Surg'],row['Peds'],row['Drug'],
#                     row['Rehab'],row['NICU'],row['LTC'],row['ICU'],row['OB'],row['Pysch'],
#                     row['Total_Beds'],row['License_Number'],row['License_exp_date'])
#     conn.commit()
#     count = count + 1
#     print(count)


# conn.commit()


# cursor.close()
# conn.close()

# Conclusion

This project has parsed a pdf and I used regex to convert the elements to a dataframe, and then I inserted into a SQL server. Hopefully you found it useful, interesting, informative, or all of the above! Thank you for reading through it and if you have any questions or comments please reach out to me.